In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv("Proceeding_Case_Data.csv" , engine='python')
df

,Number,Date,City,Fact,URL
0,٤٧١/٣/ق/,١١ / ٤ / ١٤٣٨,الدمام,الحكم الغيابي الصادر من الدائرة التجارية الراب...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13340
1,١٦٨٦/٣/ق/,١ / ١١ / ١٤٣٨,الدمام,الحكم في القضية رقم (١٦٨٦/٣/ق/ لعام ١٤٣٨ هـ)ال...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13341
2,٤٠١٢/٣/ق/,٢٥ / ١٠ / ١٤٣٨,الدمام,الحكم في القضية رقم ٤٠١٢/٣/ق/ لعام ١٤٣٢ هـالمق...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13343
3,٧١٥/٢/ق,٢١ / ٨ / ١٤٣٨,جدة,الحكم في القضية رقم ٧١٥/٢/ق لعام ١٤٣٨ هـالمقام...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13424
4,١٢٣٩١/٢/ق,١٧ / ٥ / ١٤٣٨,جدة,الحكم في القضية رقم ١٢٣٩١/٢/ق لعام ١٤٣٧هـالمقا...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13518
...,...,...,...,...,...
21607,٩٨٥٢/٣ /ق,١٢ / ٢ / ١٤٣٩,الدمام,الحكم الصادر في القضية التجارية رقم ٩٨٥٢/٣ /ق...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13601
21608,١٥٤٥/٢/ق,٢٥ / ٢ / ١٤٣٩,مكة المكرمة,الحكم الصادر في القضية رقم ١٥٤٥/٢/ق لعام ١٤٣٨ه...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13142
21609,٩٩٥٣/٣/ق,١ / ٤ / ١٤٣٩,الدمام,الحكم في القضية رقم (٩٩٥٣/٣/ق لعام ١٤٣٨هـ)المق...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13144
21610,١٠٤٦٥/٣/ق,١٧ / ٣ / ١٤٣٩,الدمام,الحكم في القضية رقم (١٠٤٦٥/٣/ق لعام ١٤٣٨هـ)الم...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13162


In [ ]:
df.isna().sum()

Number     119
Date       666
City      1426
Fact      1426
URL       2184
dtype: int64

In [ ]:
df = df.dropna(subset=['Fact'])

In [ ]:
df.isna().sum()

Number    117
Date        0
City        0
Fact        0
URL       760
dtype: int64

In [ ]:
df.duplicated().sum()

64

In [ ]:
df = df.drop_duplicates()

In [ ]:
df['Type'] = 'نظر الدعوى'

In [ ]:
# Remove diacritics
def clean_arabic_text(text):
    arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(arabic_diacritics, '', text)
    return text

df['Cleaned_Fact'] = df['Fact'].apply(clean_arabic_text)

In [ ]:
# Function to split the 'Fact' column into 'Facts', 'Reasons', and 'Judgment'
def split_fact_column(fact_text):

    facts_pattern = r'(?:الوقائع|\(الواقعات\)|\(الوقائع\)|الوقائع:\s?|\(الوقائــع\)|\(الوقائـــع\)|\(الدائرة\)|\(دائرة الاستئناف\)|\(وقائع الدعوى\)|\(واقعات الدعوى\)|\(وقائع الحكم\)|\(دائرة الاستئناف\))'

    reasons_pattern = r'(?:الأسباب:|الأسبـــاب|\(الأسباب\)|\(الأسبـــاب\)|\(أسباب الحكم\)|\(الأسباب\))'

    judgment_pattern = r'(?:حكمت\s*الدائرة|نص\s*الحكم:|حكمت\s*غيابياً|فلذلك\s*حكمت\s*الدائرة|لذلك\s*حكمت:|فلذلك\s*حكمت:|\(فلذلك\)|\(لذلك\)|\(منطوق الحكم\)|منطوق الحكم|\(قررت الدائرة\)|قررت\s*الدائرة)'


    facts_match = re.search(f'{facts_pattern}\s*(.*?)\s*{reasons_pattern}', fact_text, re.DOTALL)
    facts = facts_match.group(1).strip() if facts_match else None

    print(f'Facts Match: {facts}')

    reasons_match = re.search(f'{reasons_pattern}\s*(.*?)\s*{judgment_pattern}', fact_text, re.DOTALL)
    reasons = reasons_match.group(1).strip() if reasons_match else None

    print(f'Reasons Match: {reasons}')

    judgment_match = re.search(f'{judgment_pattern}\s*(.*)', fact_text, re.DOTALL)
    judgment = judgment_match.group(1).strip() if judgment_match else None

    print(f'Judgment Match: {judgment}')

    return facts, reasons, judgment

df[['Facts', 'Reasons', 'Judgment']] = df['Cleaned_Fact'].apply(lambda x: pd.Series(split_fact_column(x)))
df

In [ ]:
df.isna().sum()

Number           117
Date               0
City               0
Fact               0
URL              696
Type               0
Cleaned_Fact       0
Facts           1486
Reasons         1470
Judgment         705
dtype: int64

In [ ]:
df = df.dropna(subset=['Facts'])

In [ ]:
df = df.dropna(subset=['Reasons'])

In [ ]:
df.isna().sum()

Number          107
Date              0
City              0
Fact              0
URL              12
Type              0
Cleaned_Fact      0
Facts             0
Reasons           0
Judgment          0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
df = df.drop(columns=['Fact'])

In [ ]:
df = df.drop(columns=['URL'])

In [ ]:
df = df.drop(columns=['Cleaned_Fact'])